In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import pickle

In [2]:
data = pd.read_csv("data-2.tsv", sep='\t')
data

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
8224432,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
8224433,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
8224434,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
8224435,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [4]:
def clean_year(y):
    # Return year as an integer or 'NaN' if empty
    import numpy as np
    try:
        return int(y)
    except:
        return np.nan

def clean_genre(y):
    # Return only the first genre listed
    y = str(y)
    if y == '\\N':
        return ''
    return y.split(',')[0].strip()

#data.drop('endYear', axis=1, inplace=True)
data['startYear'] = data['startYear'].apply(clean_year)
data['runtimeMinutes'] = data['runtimeMinutes'].apply(clean_year)
data['genres'] = data['genres'].apply(clean_genre)
data.dropna(inplace=True, how='any', subset=['startYear', 'runtimeMinutes'])

In [5]:
mask = ((data['startYear'] >= 2016) & 
        (data['startYear'] <= 2019) &
        (data['titleType'] == 'movie') &
        (data['isAdult'] == 0) & 
        (data['runtimeMinutes'] > 80) &
        (data['genres'] != '') &
        (data['genres'] != 'Documentary'))

In [23]:
data_df = data[mask].sort_values('runtimeMinutes').head(7000)

In [24]:
data_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1490508,tt10914482,movie,Morgue,Morgue,0,2019.0,\N,81.0,Horror
6055770,tt5228262,movie,A Happening of Monumental Proportions,A Happening of Monumental Proportions,0,2017.0,\N,81.0,Comedy
5203966,tt3302092,movie,By Dawn,By Dawn,0,2019.0,\N,81.0,Drama
6059986,tt5237574,movie,Festival Film,Festival Film,0,2016.0,\N,81.0,Biography
6581595,tt6403668,movie,Karsten og Petra lager teater,Karsten og Petra lager teater,0,2017.0,\N,81.0,Family
...,...,...,...,...,...,...,...,...,...
7774637,tt8967682,movie,Love Is Life,Love Is Life,0,2019.0,\N,90.0,Comedy
6266207,tt5697990,movie,The Mover,The Mover,0,2018.0,\N,90.0,Drama
6274821,tt5717484,movie,Lost in Armenia,Celui qu'on attendait,0,2016.0,\N,90.0,Comedy
6280080,tt5729608,movie,Kamper,Kamper,0,2016.0,\N,90.0,Comedy


In [26]:
with open('movie_data.pkl', 'wb') as picklefile:
   pickle.dump(data_df['tconst'].values, picklefile)

In [4]:
import random
with open("movie_data.pkl", 'rb') as picklefile:
        links = list(pickle.load(picklefile))

In [6]:
start_urls = ['http://www.imdb.com/title/%s/' % l for l in links]

In [59]:
st_urls = start_urls[0:1000]

In [55]:
print(len(st_urls))

300


In [60]:
from time import sleep
from random import randint
import time
from IPython.core.display import clear_output

names = []
genres = []
popularity = []
director = []
runtime = []
country = []
imdb_ratings = []
metascores = []
writers = []
actors = []
writers1 = []
actors1 = []
mpaa = []
boxoffice = []
boxofficedom = []
bowknd = []
prodcomp = []
bogross = []
year = []
sound = []

# Preparing the monitoring of the loop
start_time = time.time()
requests = 0


def locate_item(soup, tag, value):
    item = soup.find(tag, attrs={'data-testid': value})
    if item is None:
        return ''
    
    return item.text

for url in st_urls:
    # Make a request 
    try:
        response = get(url)
    except requests.exceptions.RequestException as e:
        print(e)
    
    #Pause the loop
    sleep(randint(8,15))
    
    # Monitor the requests
    requests += 1
    elapsed_time = time.time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if requests > 350:
        warn('Number of requests was greater than expected.')
        break

    # Parse the content of the request with BeautifulSoup
    html_soup = BeautifulSoup(response.text, 'html.parser')
    
    
    names.append(locate_item(html_soup, 'h1', 'hero-title-block__title'))
    genres.append(locate_item(html_soup, 'div', 'genres'))
    popularity.append(locate_item(html_soup, 'div', 'hero-rating-bar__popularity__score'))
    director.append(locate_item(html_soup, 'li', 'title-pc-principal-credit'))
    runtime.append(locate_item(html_soup, 'li', 'title-techspec_runtime'))
    country.append(locate_item(html_soup, 'li', 'title-details-origin'))
    imdb_ratings.append(locate_item(html_soup, 'div', 'hero-rating-bar__aggregate-rating'))
    
    meta_=html_soup.find('span', class_='score-meta')
    if meta_ is None:
        metascores.append('')
    else:
        metascores.append(meta_.text)
        
    story_cert = html_soup.find('li', attrs = {'data-testid' :'storyline-certificate'})
    
    if story_cert is None:
        mpaa.append('')
    else:
        mpaa_rating = story_cert.find('span', class_='ipc-metadata-list-item__list-content-item')
        mpaa.append(mpaa_rating.text)

    box_office = html_soup.find('li', attrs = {'data-testid': 'title-boxoffice-budget'})
    
    if box_office is None:
        boxoffice.append('')
    else:
        bo = box_office.find('span', class_='ipc-metadata-list-item__list-content-item')
        boxoffice.append(bo.text)
        
    box_office_domestic = html_soup.find('li', attrs = {'data-testid': 'title-boxoffice-grossdomestic'})
    
    if box_office_domestic is None:
        boxofficedom.append('')
    else:
        bo_dom = box_office_domestic.find('span', class_='ipc-metadata-list-item__list-content-item')
        boxofficedom.append(bo_dom.text)
    
    box_office_wknd = html_soup.find('li', attrs = {'data-testid': 'title-boxoffice-openingweekenddomestic'})
    
    if box_office_wknd is None:
        bowknd.append('')
    else:
        bo_wknd = box_office_wknd.find('span', class_='ipc-metadata-list-item__list-content-item')
        bowknd.append(bo_wknd.text)
    
     
    box_office_gross = html_soup.find('li', attrs = {'data-testid': 'title-boxoffice-cumulativeworldwidegross'})
    
    if box_office_gross is None:
        bogross.append('')
    else:
        bo_gross = box_office_gross.find('span', class_='ipc-metadata-list-item__list-content-item')
        bogross.append(bo_gross.text)
                                         
    
    soundmix_html = html_soup.find('li', attrs = {'data-testid': 'title-techspec_soundmix'})
    
    if soundmix_html is None:
        sound.append('')
    else:
        sound.append(soundmix_html.find('a').text)                                                                              
                                         
    prod_company_html = html_soup.find('li', attrs = {'data-testid': 'title-details-companies'})
    
    if prod_company_html is None:
        prodcomp.append('')
    else:
        prod_dom = prod_company_html.find('li', class_= 'ipc-inline-list__item')
        prodcomp.append(prod_dom.text)
    
    li = html_soup.find('li', attrs={'data-testid': 'title-pc-principal-credit'})
    
    if li is not None:
        next_sib = li.find_next_sibling('li')
        if next_sib is not None:
            writer_rs = next_sib.find_all('a')
    
            if (writer_rs is not None) and (len(writer_rs)>1):
                writers1 = writer_rs[0].text
            else:
                writers1.append('')
          

            if next_sib.find_next_sibling('li') is not None:
                actor_rs = next_sib.find_next_sibling('li').find_all('a')

                if (actor_rs is not None) and (len(actor_rs)>1):
                    actors1 = actor_rs[1].text
                else:
                    actors1.append('')
            else:
                actors1.append('')
        else:
            actors1.append('')
            writers1.append('')
    else:
        actors1.append('')
        writers1.append('')
    
    writers.append(writers1)
    actors.append(actors1)
    
    actors1 = []
    writers1 = []


Request:55; Frequency: 0.07738319045602719 requests/s


In [61]:
import pandas as pd
test_df = pd.DataFrame({'movie': names,
                        'imdb': imdb_ratings,
                        'metascore': metascores,
                        'genre' : genres,
                        'popularity' : popularity,
                        'runtime' : runtime,
                        'country_of_origin' : country,
                        'director' : director,
                        'writer' : writers,
                        'actor' : actors,
                        'mpaa' : mpaa,
                        'budget' : boxoffice,
                        'box_office_dom' : boxofficedom,
                        'box_office_wknd1' : bowknd,
                        'box_office_gross': bogross,
                        'prod_company' : prodcomp,
                        'sound' : sound
                       })

test_df.to_csv("movieslist.csv")
test_df.head(5)

,movie,imdb,metascore,genre,popularity,runtime,country_of_origin,director,writer,actor,mpaa,budget,box_office_dom,box_office_wknd1,box_office_gross,prod_company,sound
0,Petmalu,IMDb RATING7.9/1014,,Comedy,,Runtime1h 30min,Country of originPhilippines,DirectorJoven Tan,[],Diego Loyzaga,,,,,,Red Nine Entertainment,
1,Roxana,IMDb RATING7.5/1019,,AnimationCrimeDrama,,Runtime1h 30min,Country of originUnited States,DirectorElahe Massumi,Anonymous,Lara Wolf,,"$1,500,000 (estimated)",,,,Roxy Films,
2,Jou Romeo,IMDb RATING6.5/1052,,ComedyRomance,,Runtime1h 30min,Country of originSouth Africa,DirectorAndre Velts,Danie Bester,Elani Dekker,,,,,"$206,112",The Film Factory,
3,Koxa,IMDb RATING4.0/10165,,CrimeDramaThriller,,Runtime1h 30min,Country of originGermany,DirectorEkrem Engizek,Burak Doruk,[],,,,,,Alpha Centauri Studios,
4,Mat Moto: Kami Mat Moto Bukan Mat Rempit,IMDb RATING2.8/1052,,ActionDrama,,Runtime1h 30min,Country of originMalaysia,DirectorsPekin IbrahimSyafiq Yusof,[],Pekin Ibrahim,,"MYR 1,000,000 (estimated)",,,,Imaginex Studios,
